In [2]:
using LinearAlgebra

Solve for $\theta$
$$y = X\theta$$

**Direct inverse**:
$$ \theta = X^{-1}y $$
Only works for square X, X may not be invertible, matrix inversion is slow and more making it useless in practice.

--------------

**Analytic solution**:
$$ \theta = (X^TX)^{-1}X^Ty $$

This can be derived by:

$$
\begin{align}
X\theta & = y \\
X^TX\theta & = X^Ty \\
\theta & = (X^TX)^{-1}X^Ty \\
\end{align}
$$

$ (X^TX)^{-1}X^T $ is equivalent to the Moore-Penrose pseudo-inverse.
`pinv` from the LinearAlgebra module provides this. 

A better "machine learning" proof by minimising the MSE cost function

$$
\begin{align}
              J(\theta) & = ||y - X\theta||_2 \\
\nabla_\theta J(\theta) & = \nabla_\theta\frac{1}{2}(X\theta - y)^T(X\theta - y)\\
                        & = \nabla_\theta\frac{1}{2}(\theta^TX^T - y^T)(X\theta - y)\\
                        & = \nabla_\theta\frac{1}{2}(\theta^TX^TX\theta - \theta^TX^Ty - y^TX\theta + y^Ty)\\
                        & = \nabla_\theta\frac{1}{2}(\theta^TX^TX\theta - 2\theta^TX^Ty + y^Ty)\\
                        & = \frac{1}{2}(2X^TX\theta - 2X^Ty)\\
                        & = X^TX\theta - X^Ty\\
             X^TX\theta & = X^Ty\\
                 \theta & = (X^TX)^{-1}X^Ty
\end{align}
$$

Notes:
* $\theta^TX^Ty$ and $y^TX\theta$ are are transposes of each other. They are also scalar valued (check the dimensions), so we can just add them up.
* The $ \frac{1}{2} $ is just added for convenience later on.
* $\nabla_\theta$ means the matrix derivative with respect to $\theta$
* Matrix differentiation rules:
$$
\begin{align}
x^TB  & \rightarrow B\\
x^Tb  & \rightarrow b\\
x^Tx  & \rightarrow 2x\\
x^TBx & \rightarrow 2Bx\\
\end{align}
$$


In [68]:
# θ = [5 2]
f(x) = 5x .+ 2
x = rand(2 ^ 12)
y = f.(x);

# include bias term
X = hcat(x, ones(size(x)));

In [4]:
function analyticsolution(X, y)
    # pinv(X) * y
    inv(X' * X) * X' * y 
end

analyticsolution (generic function with 1 method)

In [5]:
analyticsolution(X, y)

2-element Array{Float64,1}:
 5.00000000000001
 1.999999999999999

----------------

**Gradient Descent**

It's defined as
$$ \theta_j := \theta_j - \alpha \frac{\partial}{\partial \theta_j}J(\theta) $$
This just means adjust $\theta$ with respect to how much error there was.

For linear regression, we have to find out the partial derivative of the cost function with respect to each feature.
A mean squared error is an intuitive cost function to use.

Assuming there's only one training example

$$
\begin{align}
\frac{\partial}{\partial \theta_j}J(\theta) & = \frac{\partial}{\partial \theta_j} \frac{1}{2}(h_\theta(x) - y)^2\\
                                            & = 2 \cdot \frac{1}{2} (h_\theta - y) \frac{\partial}{\partial \theta_j} (h_\theta(x) - y)\\
                                            & = (h_\theta - y) \frac{\partial}{\partial \theta_j} (\sum \theta x - y)\\
                                            & = (h_\theta - y) \cdot x_j\\
\end{align}\\
$$

Plugging it into the gradient descent formula

$$ \theta_j := \theta_j - \alpha (h_\theta - y) \cdot x_j $$

For more than one training example

$$ \theta_j := \theta_j - \sum \alpha (h_\theta - y) \cdot x_j $$


Notes:
* Apply chain rule



In [6]:
function h(x, θ)
    x' * θ
end

function gradientdescent(X, y, α=0.01, tol=1e-20)
    θ = rand(size(X, 2))
    prev = θ
    diff = +Inf
    while diff > tol
        for i = 1:size(X)[1]
            θ += α * (y[i] - sum(h(X[i, :], θ))) .* X[i, :]
        end
        diff = sum((prev .- θ) .^ 2)
        prev = θ
    end
    θ
end


gradientdescent (generic function with 3 methods)

In [7]:
gradientdescent(X, y)

2-element Array{Float64,1}:
 4.9999999999995195
 2.000000000000256

--------------------

**QR Decomposition**

The QR decomposition factorizes a matrix $A$ into
* $Q$ an orthogonal matrix
* $R$ an upper triangular matrix.

Orthonormal matrices have a nice property:
$Q^TQ = QQ^T = I$ and $Q^T = Q^{-1}$

There are many ways of computing the QR decomposition. Some are
* Gram-Schmidt process
* Householder reflections
* Givens rotations

In the case of linear regression:

$$
\begin{align}
X\theta & = y\\
QR\theta & = y\\
Q^T QR & = Q^T y\\
(I)R & = Q^T y
\end{align}
$$
As $R$ is an upper triangular matrix, we can easily solve for $y$ using back-substitution.

In [63]:
# project v onto u
function project(u::AbstractVector, v::AbstractVector)
    ((v ⋅ u) / (u ⋅ u)) .* u
end


function gram_schmidt(A::AbstractMatrix)
    X = similar(A)
    X[:, 1] = A[:, 1] / norm(A[:, 1])
    for i = 2:size(X, 2)
        X[:, i] = A[:, i]
        for j = 1:i-1
            X[:, i] -= project(X[:, j], X[:, i])
        end
        X[:, i] /= norm(X[:, i])
    end
    X
end

function backward_substitution(lhs::UpperTriangular, rhs::AbstractVector)
    rows, cols = size(lhs)
    solution = ones(cols)
    solution[rows] = rhs[rows] / lhs[rows, cols]
    for i = rows-1:-1:1
        solution[i] = (rhs[i] - lhs[i, i+1:end] ⋅ solution[i+1:end]) / lhs[i, i]
    end
    solution
end

function qr_least_squares(X::AbstractMatrix, y::AbstractVector)
    Q = gram_schmidt(X)
    R = UpperTriangular(Q'X)
    backward_substitution(R, vec(Q'y))
end

qr_least_squares (generic function with 1 method)

In [69]:
qr_least_squares(X, y)

2-element Array{Float64,1}:
 5.000000000000006
 1.9999999999999991

----------------
Note that the above are just for showing how it can be computed, the easiest way to solve linear equations is in Julia is

In [71]:
X \ y

2-element Array{Float64,1}:
 5.000000000000001
 1.9999999999999987